In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
def load_images_from_folder(dataset_path, img_size=(256, 256)):
    images, masks = [], []
    categories = ['benign', 'normal', 'malignant']

    for category in categories:
        category_path = os.path.join(dataset_path, category)
        image_files = sorted([f for f in os.listdir(category_path) if "mask" not in f.lower()])
        mask_files = sorted([f for f in os.listdir(category_path) if "mask" in f.lower()])

        for img_file, mask_file in zip(image_files, mask_files):
            img_path, mask_path = os.path.join(category_path, img_file), os.path.join(category_path, mask_file)
            img, mask = cv2.imread(img_path), cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if img is not None and mask is not None:
                images.append(cv2.resize(img, img_size) / 255.0)
                masks.append(cv2.resize(mask, img_size) / 255.0)

    return np.array(images), np.expand_dims(np.array(masks), axis=-1)

dataset_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"
train_images, train_masks = load_images_from_folder(dataset_path)
test_images, test_masks = load_images_from_folder(dataset_path)

print("Train Images Shape:", train_images.shape)
print("Train Masks Shape:", train_masks.shape)
print("Test Images Shape:", test_images.shape)
print("Test Masks Shape:", test_masks.shape)

In [ ]:
def build_model(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs)

    encoder_output = base_model.output  
    
    # **Decoder (Upsampling Layers)**
    x = Conv2DTranspose(256, (3,3), strides=(2,2), padding="same")(encoder_output)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(128, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(64, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(32, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2DTranspose(16, (3,3), strides=(2,2), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(1, (1,1), activation="sigmoid")(x)

    model = Model(inputs, x)
    return model

unet_model = build_model()
unet_model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
unet_model.summary()

In [ ]:
history = unet_model.fit(
    train_images, train_masks,
    validation_data=(test_images, test_masks),
    epochs=50,
    batch_size=16,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
def plot_sample(image, predicted_mask, actual_mask, index):
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title(f'Original Image {index}')
    ax[0].axis('off')

    ax[1].imshow(predicted_mask, cmap='viridis')
    ax[1].set_title(f'Predicted Mask {index}')
    ax[1].axis('off')

    ax[2].imshow(actual_mask, cmap='viridis')
    ax[2].set_title(f'Actual Mask {index}')
    ax[2].axis('off')

    plt.show()

# Predict and visualize
predictions = unet_model.predict(test_images)
predictions = (predictions > 0.5).astype(np.uint8)  # Binarize output

for i in range(3):  # Show 3 samples
    plot_sample(test_images[i], predictions[i].squeeze(), test_masks[i].squeeze(), i)